## **Simple data cleaning**



After annotating the SAS script code (getting the matching instructions using GPT 3.5 API) we proceed to clean anf prepare our dataset for the fine tuning process

In [1]:
import pandas as pd
parquet_file = r'/content/df_annotated_finalbatch.parquet'
df=pd.read_parquet(parquet_file, engine = 'pyarrow')
df

,content,Annotation
0,"libname storm xlsx ""/home/student/Courses/gasa...","""Set the library 'storm' to the Excel file loc..."
1,/* Advent of Code Day 2 Puzzle 2 */\n\nDATA da...,"Create a data frame named ""day2"" by reading in..."
2,%macro Ulcer_Index_test(keep=FALSE);\n%global ...,"I am sorry, but I cannot provide an instructio..."
3,FILENAME REFFILE '/folders/myshortcuts/SASUniv...,"""Import the CSV file located at '/folders/mysh..."
4,"\nlibname b ""./"";\n\nproc contents data=b.pers...",Please create a program that accomplishes the ...
...,...,...
11383,**********************************************...,The instruction to match the SAS programming l...
11384,"/* \nFrom: Xie, Fenglong [mailto:fenglongxie@u...","Create a macro named ""ciras"" that takes in thr..."
11385,/*********************************************...,Create SAS informats for missing values using ...
11386,"proc iml;\n\n\tstart m(t) global(c1,c2,age);\n...","Sorry, I cannot provide an instruction for an ..."


We want to get rid of every wrong annotation that would harm the efficieny of our models, therefore we propose as an initial step to remove any non generated annotation (ie any type of 'Sorry, cannot provide an instruction ..')

In [2]:
df['Annotation_lower'] = df['Annotation'].str.lower()
keywords = ['Sorry', 'Cannot provide']
# Ensure keywords are lowercase
keywords = [keyword.lower() for keyword in keywords]

filtered_df = df[df['Annotation_lower'].str.contains('|'.join(keywords))]
filtered_df


,content,Annotation,Annotation_lower
2,%macro Ulcer_Index_test(keep=FALSE);\n%global ...,"I am sorry, but I cannot provide an instructio...","i am sorry, but i cannot provide an instructio..."
7,*Ex24_Week10_List_of_Files.sas;\ntitle; title1...,"Sorry, as an AI language model, I cannot provi...","sorry, as an ai language model, i cannot provi..."
21,begin_version\n3.FOND\nend_version\nbegin_metr...,"I'm sorry, but it is not possible to provide a...","i'm sorry, but it is not possible to provide a..."
27,%macro Prospect_Ratio_test(keep=FALSE);\n%glob...,"I'm sorry, as an AI language model, I cannot p...","i'm sorry, as an ai language model, i cannot p..."
30,%macro table_CalendarReturns_test2(keep=FALSE)...,"I'm sorry, as an AI language model, I cannot p...","i'm sorry, as an ai language model, i cannot p..."
...,...,...,...
11358,%macro return_calculate_test2(keep=FALSE);\n%g...,"I'm sorry, I cannot provide an LLM model instr...","i'm sorry, i cannot provide an llm model instr..."
11371,/*********************************************...,"Sorry, as an AI language model, I am not able ...","sorry, as an ai language model, i am not able ..."
11372,/*data example; input batch percent @@; cards;...,"Sorry, as an AI language model, I do not have ...","sorry, as an ai language model, i do not have ..."
11380,%macro return_centered_test(keep=FALSE);\n%glo...,"I am sorry, but it is not clear what task you ...","i am sorry, but it is not clear what task you ..."


In [3]:
filtered_df['Annotation'].iloc[0]

'I am sorry, but I cannot provide an instruction for an LLM model to match this SAS programming language script as LLM models are used for natural language processing and cannot replicate programming language scripts.'

We can attempt to regenerate annotations for these lines since they seem to have misinterpreted our intentions, likely due to the prompt used for annotation. However, considering the time constraints of the project and the negligible number of rows affected by this issue, we could simply delete these lines.

In [4]:
# Select rows that do NOT contain any of the keywords
cleaned_df = df[~df['Annotation_lower'].str.contains('|'.join(keywords))]
cleaned_df

,content,Annotation,Annotation_lower
0,"libname storm xlsx ""/home/student/Courses/gasa...","""Set the library 'storm' to the Excel file loc...","""set the library 'storm' to the excel file loc..."
1,/* Advent of Code Day 2 Puzzle 2 */\n\nDATA da...,"Create a data frame named ""day2"" by reading in...","create a data frame named ""day2"" by reading in..."
3,FILENAME REFFILE '/folders/myshortcuts/SASUniv...,"""Import the CSV file located at '/folders/mysh...","""import the csv file located at '/folders/mysh..."
4,"\nlibname b ""./"";\n\nproc contents data=b.pers...",Please create a program that accomplishes the ...,please create a program that accomplishes the ...
5,More precise computations at macro excecution ...,Perform more precise calculations at macro exe...,perform more precise calculations at macro exe...
...,...,...,...
11382,/*--------------------------------------------...,Provide a demo of MISSOVER in a SAS program. U...,provide a demo of missover in a sas program. u...
11383,**********************************************...,The instruction to match the SAS programming l...,the instruction to match the sas programming l...
11384,"/* \nFrom: Xie, Fenglong [mailto:fenglongxie@u...","Create a macro named ""ciras"" that takes in thr...","create a macro named ""ciras"" that takes in thr..."
11385,/*********************************************...,Create SAS informats for missing values using ...,create sas informats for missing values using ...


## **Tokenization for precise price estimation of annotation**

Let's try to tokenize our SAS programming language using the code.tokenize python library

In [5]:
!pip install code-tokenize

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.7/496.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00
  Created wheel for code-tokenize: filename=code_tokenize-0.2.0-py3-none-any.whl size=13860 sha256=db00e222c6a72b7cf6313f663a2fcf6810530bb811ec43fe0e01e7fd127461a8
  Stored in directory: /root/.cache/pip/wheels/8b/20/d2/c85b7294f3aa192ade2924a7e2c94c6b0f45c806c181644b64
  Created wheel for code-ast: filename=code_ast-0.1.0-py3-none-any.whl size=11828 sha256=eb18670acb1443bcdbc54dbdcd65bad87cea894f7c5db81bf8be704814d0a9c9
  Stored in directory: /root/.cache/pip/wheels/39/6c/09/e08b77926bdbe612cc11e7d460f2d35aa646d5fbdf94f85933
Successfully built code-tokenize code-ast


In [6]:
import code_tokenize as ctok

In [7]:
selected_row = cleaned_df.loc[0, 'content']

# Tokenize the selected SAS code
# Replace 'python' with the correct language if SAS is supported
tokens = ctok.tokenize(selected_row, lang="python", syntax_error="ignore")

print("Tokens:", tokens)
len(tokens)

/usr/local/lib/python3.10/dist-packages/tree_sitter/__init__.py:36: FutureWarning: Language.build_library is deprecated. Use the new bindings instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


Tokens: [libname, storm, xlsx, "/home/student/Courses/gasas/Storm.xlsx", #NEWLINE#, ;, proc, contents, data, =, storm, ., _all_, #NEWLINE#, nods, ;, run, #NEWLINE#, ;, libname, gasas, "/home/student/Courses/gasas", #NEWLINE#, ;, proc, copy, inlib, =, storm, outlib, =, gasas, #NEWLINE#, ;, run, #NEWLINE#, ;]


/usr/local/lib/python3.10/dist-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


37

In [8]:
cleaned_df['Tokens'] = cleaned_df['content'].apply(lambda x: ctok.tokenize(x, lang="python", syntax_error="ignore"))

# Count the number of tokens for each script
cleaned_df['Token_Count'] = cleaned_df['Tokens'].apply(len)
cleaned_df

<ipython-input-8-fca695c50235>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Tokens'] = cleaned_df['content'].apply(lambda x: ctok.tokenize(x, lang="python", syntax_error="ignore"))
<ipython-input-8-fca695c50235>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Token_Count'] = cleaned_df['Tokens'].apply(len)


,content,Annotation,Annotation_lower,Tokens,Token_Count
0,"libname storm xlsx ""/home/student/Courses/gasa...","""Set the library 'storm' to the Excel file loc...","""set the library 'storm' to the excel file loc...","[libname, storm, xlsx, ""/home/student/Courses/...",37
1,/* Advent of Code Day 2 Puzzle 2 */\n\nDATA da...,"Create a data frame named ""day2"" by reading in...","create a data frame named ""day2"" by reading in...","[/, *, Advent, of, Code, Day, 2, Puzzle, 2, *,...",131
3,FILENAME REFFILE '/folders/myshortcuts/SASUniv...,"""Import the CSV file located at '/folders/mysh...","""import the csv file located at '/folders/mysh...","[FILENAME, REFFILE, '/folders/myshortcuts/SASU...",541
4,"\nlibname b ""./"";\n\nproc contents data=b.pers...",Please create a program that accomplishes the ...,please create a program that accomplishes the ...,"[libname, b, ""./"", #NEWLINE#, ;, proc, content...",266
5,More precise computations at macro excecution ...,Perform more precise calculations at macro exe...,perform more precise calculations at macro exe...,"[More, precise, computations, at, macro, excec...",559
...,...,...,...,...,...
11382,/*--------------------------------------------...,Provide a demo of MISSOVER in a SAS program. U...,provide a demo of missover in a sas program. u...,"[/, *, -, -, -, -, -, -, -, -, -, -, -, -, -, ...",925
11383,**********************************************...,The instruction to match the SAS programming l...,the instruction to match the sas programming l...,"[*, *, *, *, *, *, *, *, *, *, *, *, *, *, *, ...",758
11384,"/* \nFrom: Xie, Fenglong [mailto:fenglongxie@u...","Create a macro named ""ciras"" that takes in thr...","create a macro named ""ciras"" that takes in thr...","[/, *, From, :, Xie, ,, Fenglong, [, mailto, :...",809
11385,/*********************************************...,Create SAS informats for missing values using ...,create sas informats for missing values using ...,"[, /, *, *, *, *, *, *, *, *, *, *, *, *, *, *...",1480


In [9]:
max_token_count = cleaned_df['Token_Count'].max()

print("Maximum Token Count:", max_token_count)


Maximum Token Count: 10759


In [10]:
cleaned_df['Token_Count'].describe()

count    10909.000000
mean       750.249702
std        620.457866
min          5.000000
25%        266.000000
50%        647.000000
75%       1047.000000
max      10759.000000
Name: Token_Count, dtype: float64

This allows us to have more precise estimation of the price of annotation even though we didn't use a SAS tokenizer as we didn't find one for now. The estimation is updated in the Coût_annotation_SAS.xlsx file.

## **Formatting the dataset for Mistral-Instruct**

In order to leverage instruction fine-tuning, the prompt should be surrounded by [INST] and [/INST] tokens. The very first instruction should begin with a begin of sentence id. The next instructions should not. The assistant generation will be ended by the end-of-sentence token id.

EG : text = "\<s>[INST] What is your favourite condiment? [/INST]"
"Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!\</s> "
"[INST] Do you have mayonnaise recipes? [/INST]"

In [11]:
df = cleaned_df[['content', 'Annotation']]
df

,content,Annotation
0,"libname storm xlsx ""/home/student/Courses/gasa...","""Set the library 'storm' to the Excel file loc..."
1,/* Advent of Code Day 2 Puzzle 2 */\n\nDATA da...,"Create a data frame named ""day2"" by reading in..."
3,FILENAME REFFILE '/folders/myshortcuts/SASUniv...,"""Import the CSV file located at '/folders/mysh..."
4,"\nlibname b ""./"";\n\nproc contents data=b.pers...",Please create a program that accomplishes the ...
5,More precise computations at macro excecution ...,Perform more precise calculations at macro exe...
...,...,...
11382,/*--------------------------------------------...,Provide a demo of MISSOVER in a SAS program. U...
11383,**********************************************...,The instruction to match the SAS programming l...
11384,"/* \nFrom: Xie, Fenglong [mailto:fenglongxie@u...","Create a macro named ""ciras"" that takes in thr..."
11385,/*********************************************...,Create SAS informats for missing values using ...


We save this cleaned version as a Parquet file at first, we might need it to fine tune other models other than Mistral-Instruct.

In [12]:
# Specifying the file path where we want to save the DataFrame
parquet_file_path = '/content/df_annotated_cleaned.parquet'

# Save the DataFrame to a Parquet file
df.to_parquet(parquet_file_path, index=False)

In [13]:
import json

In [ ]:
# Initialize the formatted data list
formatted_data = []

# Iterate over the DataFrame
for index, row in df.iterrows():
    if index == 0:
        # First instruction and response, with <s> at the start and </s> at the end
        formatted_row = f'<s>[INST] {row["Annotation"]} [/INST] "{row["content"]}"</s>'
    else:
        # Subsequent instructions and responses without <s> or </s>
        formatted_row = f'[INST] {row["Annotation"]} [/INST] "{row["content"]}"'

    formatted_data.append(formatted_row)

# Join all pieces into a single string
formatted_string = " ".join(formatted_data)

# Save to JSON
file_path = '/content/instructions_data_mistral.json'
with open(file_path, 'w') as outfile:
    json.dump({'text': formatted_string}, outfile)

print(f"Data has been correctly formatted and saved to {file_path}")


Data has been correctly formatted and saved to /content/instructions_data_mistral.json


In [ ]:
file_path = '/content/formatted_instructions.jsonl'

with open(file_path, 'w') as outfile:
    for index, row in df.iterrows():
        # Format the instruction and response according to the given structure
        formatted_text = f'<s>[INST] {row["Annotation"]} [/INST] {row["content"]} </s>'

        # Write each formatted string as a separate line in the .jsonl file
        json.dump({"text": formatted_text}, outfile)
        outfile.write('\n')  # Ensure each JSON object is on a new line

print(f"Data has been formatted and saved to {file_path}")

Data has been formatted and saved to /content/formatted_instructions.jsonl
